In [1]:
# 1) IMPORTS
import pandas as pd
import glob, re

# 2) LOAD & CONCAT ALL WORLD CUP CSVs
files = [f for f in glob.glob('data/FIFA - *.csv')
         if re.match(r'.*FIFA - \d{4}\.csv$', f)]
dfs = []
for fn in files:
    year = int(re.search(r'FIFA - (\d{4})\.csv', fn).group(1))
    tmp = pd.read_csv(fn)
    tmp['year'] = year
    dfs.append(tmp)
df = pd.concat(dfs, ignore_index=True)

# 3) CLEAN COLUMN NAMES & TYPES
df.columns = [c.strip() for c in df.columns]
for col in ['Position','Games Played','Win','Draw','Loss',
            'Goals For','Goals Against','Goal Difference','Points']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4) STANDARDIZE TEAM NAMES
former = pd.read_csv('data/former_names.csv')
mapping = dict(zip(former['former'], former['current']))
df['team'] = (
    df['Team']
      .replace(mapping)
      .str.replace(r'\*+', '', regex=True)
      .str.strip()
)

# 5) MAP CONFEDERATIONS
ranks = pd.read_csv('data/fifa_ranking-2024-06-20.csv')
ranks['rank_date'] = pd.to_datetime(ranks['rank_date'])
latest = (
    ranks
    .sort_values('rank_date')
    .drop_duplicates('country_full', keep='last')
)
conf_map = dict(zip(latest['country_full'], latest['confederation']))
manual = {
    'South Korea':'AFC','Iran':'AFC','United States':'CONCACAF',
    'Ivory Coast':'CAF','Czech Republic':'UEFA','North Korea':'AFC',
    'West Germany':'UEFA','East Germany':'UEFA','FR Yugoslavia':'UEFA',
    'Dutch East Indies':'AFC','Israel':'UEFA','Bulgaria':'UEFA'
}
df['confed'] = df['team'].map(conf_map)
df['confed'] = df.apply(
    lambda r: manual.get(r['team'], r['confed']), axis=1
)

# 6) COMPUTE SUCCESS METRICS
def stage_score(pos):
    if pos == 1:      return 6
    if pos == 2:      return 5
    if pos in (3,4):  return 4
    if pos <= 8:      return 3
    if pos <= 16:     return 2
    return 1

df['stage_score']  = df['Position'].astype(int).apply(stage_score)
df['win_rate']     = df['Win']   / df['Games Played']
df['gd_per_game']  = df['Goal Difference'] / df['Games Played']
df['gf_per_game']  = df['Goals For']       / df['Games Played']

# 7) SAVE PER-TOURNAMENT MASTER
df.to_csv('output/worldcup_master.csv', index=False)

# 8) BUILD TEAM-LEVEL SUMMARY
# a) Full-history sum of stage_score
team_full = (
    df
    .groupby('team')['stage_score']
    .sum()
    .reset_index(name='sum_stage_score_full')
)

# b) Post-1994 sum of stage_score
team_post94 = (
    df[df['year'] >= 1994]
    .groupby('team')['stage_score']
    .sum()
    .reset_index(name='sum_stage_score_post94')
)

# c) Average FIFA rank (1994+)
ranks['year'] = ranks['rank_date'].dt.year
team_rank = (
    ranks[ranks['year'] >= 1994]
    .groupby('country_full')['rank']
    .mean()
    .reset_index(name='avg_rank')
)

# d) Secondary metric averages (full history)
team_sec = (
    df
    .groupby('team')[['win_rate','gd_per_game','gf_per_game']]
    .mean()
    .reset_index()
)

# e) Confederation per team
team_conf = (
    df[['team','confed']]
    .drop_duplicates('team')
)

# f) Merge all components
team_stats = (
    team_full
    .merge(team_post94, on='team', how='outer')
    .merge(team_sec,     on='team', how='left')
    .merge(team_rank,    left_on='team', right_on='country_full', how='left')
    .merge(team_conf,    on='team', how='left')
    [['team','confed',
      'sum_stage_score_full','sum_stage_score_post94',
      'win_rate','gd_per_game','gf_per_game',
      'avg_rank']]
)

# g) SAVE TEAM STATS
team_stats.to_csv('output/team_stats.csv', index=False)
